# 1. Import Libraries and Datasets

In [1]:
# Import libraries
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown
from sklearn.model_selection import train_test_split

In [2]:
# only use the train series and train events dataset in the kaggle competition
df_train_series = pd.read_parquet('train_series.parquet')
df_train_events = pd.read_csv('train_events.csv')

# 2. Data Preprocessing

## 2.1 Showcase the basic information of datasets

In [3]:
df_train_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127946340 entries, 0 to 127946339
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  object 
 1   step       uint32 
 2   timestamp  object 
 3   anglez     float32
 4   enmo       float32
dtypes: float32(2), object(2), uint32(1)
memory usage: 3.3+ GB


In [7]:
df_train_series['series_id'].nunique()

277

In [4]:
df_train_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14508 entries, 0 to 14507
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   series_id  14508 non-null  object 
 1   night      14508 non-null  int64  
 2   event      14508 non-null  object 
 3   step       9585 non-null   float64
 4   timestamp  9585 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 566.8+ KB


From the information shown above, we can see that the datasets are very huge and occupy a lot of memory space, which may cause large RAM requirement during the implementation. Therefore, our first step is to reduce the memory.

## 2.2 Reduce the Size of Datasets

### 2.2.1 Reduce the Memory Usage by reducing rows

In [5]:
new_series = df_train_series.copy()
new_events = df_train_events.copy()

In [6]:
# see JustDeleteNanLine.ipynb figure 1 to understand why we need this step
# delete those series_id that have NaN values
# Group the series_id and check the step column of each group for NaN values
fully_coverage_series_ids = new_events.groupby('series_id').apply(lambda x: x['step'].isnull().sum() == 0)

# only choose those series_id that have no NaN values
fully_coverage_series_ids = fully_coverage_series_ids.index[fully_coverage_series_ids].tolist()

len(fully_coverage_series_ids)

37

In [7]:
# Filter train_series to keep only the series_id present in fully_coverage_series_ids
new_series = new_series[new_series['series_id'].isin(fully_coverage_series_ids)]

In [8]:
new_series.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14334840 entries, 3770820 to 103353659
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  object 
 1   step       uint32 
 2   timestamp  object 
 3   anglez     float32
 4   enmo       float32
dtypes: float32(2), object(2), uint32(1)
memory usage: 492.1+ MB


In [9]:
# Select the series_id in the train_events file in fully_coverage_series_ids
new_events = new_events[new_events['series_id'].isin(fully_coverage_series_ids)]

In [10]:
new_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1562 entries, 398 to 11677
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   series_id  1562 non-null   object 
 1   night      1562 non-null   int64  
 2   event      1562 non-null   object 
 3   step       1562 non-null   float64
 4   timestamp  1562 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 73.2+ KB


By now, we can see that we've decreased the train_series from 3.3 GB to 492.1 MB, and the train_events from 566.8 KB to 73.2 KB. Next, we will try to further reducing the datasets by changing the data types.

### 2.2.2 Further Reducing the dataset by changing the data types

First, we map the unique 37 values of series_id from the original values to 0-36, and then we change the datatype from object to uint8, which will decrease the memory usage.

In [11]:
# create the mapping from series_id to new id from 0 to 36
unique_series_ids = new_series['series_id'].unique()
id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_series_ids)}

# map the series_id to new id
new_series['series_id'] = new_series['series_id'].map(id_mapping)

# change data type to uint8 to save memory
new_series['series_id'] = new_series['series_id'].astype('uint8')

In [12]:
new_series.head()

,series_id,step,timestamp,anglez,enmo
3770820,0,0,2018-11-05T10:00:00-0400,-30.845301,0.0447
3770821,0,1,2018-11-05T10:00:05-0400,-34.181801,0.0443
3770822,0,2,2018-11-05T10:00:10-0400,-33.877102,0.0483
3770823,0,3,2018-11-05T10:00:15-0400,-34.282101,0.0680
3770824,0,4,2018-11-05T10:00:20-0400,-34.385799,0.0768


In [13]:
tqdm.pandas()

# convert timestamp to datetime and show the progress bar using tqdm
new_series['timestamp'] = new_series['timestamp'].progress_apply(pd.to_datetime)

100%|██████████| 14334840/14334840 [14:13<00:00, 16797.69it/s]


In [14]:
# Apply mapping to update series_id in df_train_events
new_events['series_id'] = new_events['series_id'].map(id_mapping)

# Convert the data type of series_id to uint8
new_events['series_id'] = new_events['series_id'].astype('uint8')


In [15]:
new_events.head()

,series_id,night,event,step,timestamp
398,0,1,onset,11988.0,2018-11-06T02:39:00-0400
399,0,1,wakeup,14388.0,2018-11-06T05:59:00-0400
400,0,2,onset,28680.0,2018-11-07T01:50:00-0400
401,0,2,wakeup,31320.0,2018-11-07T05:30:00-0400
402,0,3,onset,44184.0,2018-11-07T23:22:00-0400


In [16]:
tqdm.pandas()

# convert datatype 
# using one-hot encoding to convert the event column to 0(onset) and 1(wakeup)
new_events['night'] = new_events['night'].astype(np.uint16)
new_events['event'] = new_events['event'].replace({'onset':'0', 'wakeup':'1'}).astype(np.uint8)
new_events['step'] = new_events['step'].astype(np.uint32)

In [17]:
# save to csv file
new_events.to_csv('new_events.csv', index=False)

In [18]:
new_events.head()

,series_id,night,event,step,timestamp
398,0,1,0,11988,2018-11-06T02:39:00-0400
399,0,1,1,14388,2018-11-06T05:59:00-0400
400,0,2,0,28680,2018-11-07T01:50:00-0400
401,0,2,1,31320,2018-11-07T05:30:00-0400
402,0,3,0,44184,2018-11-07T23:22:00-0400


In [19]:
new_series.head()

,series_id,step,timestamp,anglez,enmo
3770820,0,0,2018-11-05 10:00:00-04:00,-30.845301,0.0447
3770821,0,1,2018-11-05 10:00:05-04:00,-34.181801,0.0443
3770822,0,2,2018-11-05 10:00:10-04:00,-33.877102,0.0483
3770823,0,3,2018-11-05 10:00:15-04:00,-34.282101,0.0680
3770824,0,4,2018-11-05 10:00:20-04:00,-34.385799,0.0768


## 2.3 Merge the Datasets

In [20]:
# Add hour as a new column to train_series
tqdm.pandas()
new_series['hour'] = new_series['timestamp'].progress_apply(lambda x: x.hour)

100%|██████████| 14334840/14334840 [00:10<00:00, 1343017.48it/s]


In [21]:
# Based on the series_id and step in train_events, add event and night to train_series
def get_event_or_night(series, events, column_name):
    matched_event = events[(events['series_id'] == series['series_id']) & (events['step'] == series['step'])][column_name]
    return matched_event.values[0] if len(matched_event) > 0 else np.nan  # return NaN if no match

new_series['event'] = new_series.progress_apply(lambda x: get_event_or_night(x, new_events, 'event'), axis=1)
new_series['night'] = new_series.progress_apply(lambda x: get_event_or_night(x, new_events, 'night'), axis=1)


100%|██████████| 14334840/14334840 [1:28:57<00:00, 2685.80it/s]


In [22]:
new_series.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14334840 entries, 3770820 to 103353659
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  uint8  
 1   step       uint32 
 2   timestamp  object 
 3   anglez     float32
 4   enmo       float32
 5   hour       int64  
 6   event      float64
 7   night      float64
dtypes: float32(2), float64(2), int64(1), object(1), uint32(1), uint8(1)
memory usage: 724.6+ MB


In [23]:
# chech how many NaN values in event and night column
new_series['event'].isnull().sum()
new_series['night'].isnull().sum()


14333278

In [24]:
new_series_copy = new_series.copy()

In [25]:
# Fill the null values of event and night according to the before and after values
new_series['event'] = new_series['event'].fillna(method='ffill').fillna(method='bfill')
new_series['night'] = new_series['night'].fillna(method='ffill').fillna(method='bfill')

In [26]:
# check again how many NaN values in event and night column
new_series['event'].isnull().sum()
new_series['night'].isnull().sum()

0

In [27]:
new_series.head()

,series_id,step,timestamp,anglez,enmo,hour,event,night
3770820,0,0,2018-11-05 10:00:00-04:00,-30.845301,0.0447,10,0.0,1.0
3770821,0,1,2018-11-05 10:00:05-04:00,-34.181801,0.0443,10,0.0,1.0
3770822,0,2,2018-11-05 10:00:10-04:00,-33.877102,0.0483,10,0.0,1.0
3770823,0,3,2018-11-05 10:00:15-04:00,-34.282101,0.0680,10,0.0,1.0
3770824,0,4,2018-11-05 10:00:20-04:00,-34.385799,0.0768,10,0.0,1.0


Finally, we merge the train_series and train_events into one dataset, and we can see that the memory usage is 724.6 MB, which is much smaller than the original datasets(3.3 GB). And we add some columns to the datasets, which are important steps for the following data analysis.

The final size of new_series is 724.6MB, merging train_events and train_series.

In [28]:
new_series.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14334840 entries, 3770820 to 103353659
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  uint8  
 1   step       uint32 
 2   timestamp  object 
 3   anglez     float32
 4   enmo       float32
 5   hour       int64  
 6   event      float64
 7   night      float64
dtypes: float32(2), float64(2), int64(1), object(1), uint32(1), uint8(1)
memory usage: 724.6+ MB


In [29]:
# new_series.to_parquet('new_series.parquet', index=False)
# no need to run this line again, because we have already saved the new_series.parquet file